<a href="https://colab.research.google.com/github/rpjena/random_matrix/blob/main/ipcarud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
from scipy.optimize import minimize

# Define the dimensions
N = 100  # Number of firms
L = 5    # Number of characteristics
T = 50   # Number of time periods

# Generate random data for demonstration
np.random.seed(42)
Z = np.random.randn(T-1, N, L)  # Characteristics matrix
r = np.random.randn(T-1, N)     # Returns matrix

# Initialize Gamma_beta and f
Gamma_beta = np.random.randn(L, L)
f = np.random.randn(T-1, L)

def objective_function(params, Z, r):
    Gamma_beta = params['Gamma_beta']
    f = params['f']
    total_error = 0
    for t in range(T-1):
        error = r[t] - Z[t] @ Gamma_beta @ f[t]
        total_error += np.sum(error ** 2)
    return total_error

def update_f(Gamma_beta, Z, r):
    f_new = np.zeros_like(f)
    for t in range(T-1):
        term1 = Gamma_beta.T @ Z[t].T @ Z[t] @ Gamma_beta
        term2 = Gamma_beta.T @ Z[t].T @ r[t]
        f_new[t] = np.linalg.inv(term1) @ term2
    return f_new

def update_Gamma_beta(f, Z, r):
    term1 = np.zeros((L*L, L*L))
    term2 = np.zeros(L*L)
    for t in range(T-1):
        term1 += np.kron(Z[t].T @ Z[t], np.outer(f[t], f[t]))
        term2 += np.kron(Z[t], f[t].T).T @ r[t]
    Gamma_beta_new = np.linalg.inv(term1) @ term2
    return Gamma_beta_new.reshape(L, L)

# Alternating least squares
max_iter = 100
tolerance = 1e-6
for iteration in range(max_iter):
    f = update_f(Gamma_beta, Z, r)
    Gamma_beta_new = update_Gamma_beta(f, Z, r)

    # Check for convergence
    if np.linalg.norm(Gamma_beta_new - Gamma_beta) < tolerance:
        break
    Gamma_beta = Gamma_beta_new

print(f"Converged after {iteration} iterations")
print("Estimated Gamma_beta:\n", Gamma_beta)
print("Estimated f:\n", f)

Converged after 0 iterations
Estimated Gamma_beta:
 [[ 1.2341168  -0.14660706 -0.40039734 -1.9517073  -0.64011277]
 [-0.77168646 -0.79477309  0.33283614  0.12224855 -1.36129593]
 [-1.2978411  -1.29967855  1.81052609  1.49272263 -0.852299  ]
 [-1.97659432  0.12804424  0.04694012  0.31691699  0.74137389]
 [ 0.45170395 -1.11643953  0.29404109  0.85382358 -0.72400172]]
Estimated f:
 [[-0.02797797 -0.09917325 -0.09759797  0.01674758  0.10031647]
 [ 0.09469515  0.27341618  0.15811726  0.00674043 -0.13400086]
 [-0.02482132 -0.13584852 -0.06539255 -0.08212917  0.06979606]
 [ 0.01372863 -0.05994633  0.06779654  0.00090025  0.12182501]
 [-0.00192829  0.24001124  0.06695903  0.03812078 -0.0831417 ]
 [ 0.10977443  0.10793889  0.11851746  0.03330085 -0.06837653]
 [ 0.01028998 -0.04131726 -0.09396054  0.04980053  0.01174087]
 [-0.03315704 -0.1787283   0.01335608 -0.11245052  0.10837513]
 [-0.03947426 -0.21036761 -0.24288196  0.00488569 -0.01834831]
 [ 0.11527402  0.05233919  0.11767856  0.0536038   

In [8]:
import numpy as np

# Define the dimensions
N = 100  # Number of firms
L = 5    # Number of characteristics
T = 50   # Number of time periods
K = 3    # Number of factors (assumed based on the context)

# Generate random data for demonstration
np.random.seed(42)
Z = np.random.randn(T-1, N, L)  # Characteristics matrix (T-1 x N x L)
r = np.random.randn(T-1, N)     # Returns matrix (T-1 x N)

# Compute managed portfolio returns x_t+1 = Z_t' r_t+1 / N_t+1
X = np.zeros((T-1, L))  # T-1 x L
for t in range(T-1):
    X[t] = Z[t].T @ r[t] / N  # L x 1

# Compute the sample second moment matrix of managed portfolio returns
X_prime_X = X.T @ X  # L x L

# Initialize Gamma_beta as the first K eigenvectors of X'X
eigenvalues, eigenvectors = np.linalg.eig(X_prime_X)
Gamma_beta = eigenvectors[:, :K]  # L x K

# Initialize f as random (T-1 x K)
f = np.random.randn(T-1, K)

def update_f(Gamma_beta, Z, r):
    f_new = np.zeros_like(f)  # T-1 x K
    for t in range(T-1):
        # Z[t] is N x L, Gamma_beta is L x K, r[t] is N x 1
        term1 = Gamma_beta.T @ Z[t].T @ Z[t] @ Gamma_beta  # K x K
        term2 = Gamma_beta.T @ Z[t].T @ r[t]  # K x 1
        f_new[t] = np.linalg.inv(term1) @ term2  # K x 1
    return f_new

def update_Gamma_beta(f, Z, r):
    term1 = np.zeros((L*K, L*K))  # L*K x L*K
    term2 = np.zeros(L*K)         # L*K x 1
    for t in range(T-1):
        # Z[t].T @ Z[t] is L x L, np.outer(f[t], f[t]) is K x K
        term1 += np.kron(Z[t].T @ Z[t], np.outer(f[t], f[t]))  # L*K x L*K
        # Z[t].T is L x N, f[t].T is 1 x K, r[t] is N x 1
        term2 += np.kron(Z[t], f[t].T).T @ r[t]  # L*K x 1
    Gamma_beta_new = np.linalg.inv(term1) @ term2  # L*K x 1
    return Gamma_beta_new.reshape(L, K)  # Reshape to L x K

# Alternating least squares
max_iter = 100
tolerance = 1e-6
for iteration in range(max_iter):
    f_new = update_f(Gamma_beta, Z, r)
    Gamma_beta_new = update_Gamma_beta(f_new, Z, r)

    # Check for convergence
    if np.linalg.norm(Gamma_beta_new - Gamma_beta) < tolerance:
        break
    Gamma_beta = Gamma_beta_new
    f = f_new

print(f"Converged after {iteration} iterations")
print("Estimated Gamma_beta:\n", Gamma_beta)
print("Estimated f:\n", f)

Converged after 73 iterations
Estimated Gamma_beta:
 [[-0.03174403  0.08268427 -0.19357203]
 [-1.05319822  0.25329806 -0.40962646]
 [ 0.10505486 -0.08822077 -0.56261932]
 [ 0.34513083  0.92875924 -0.19584552]
 [ 0.31461493 -0.1983074  -0.76345298]]
Estimated f:
 [[ 0.0801501   0.10515259  0.03564492]
 [-0.05944509 -0.18642835  0.13682296]
 [ 0.00151613  0.08316311 -0.01133644]
 [ 0.1062038   0.01723252 -0.00919054]
 [-0.01249717  0.01939661  0.17448965]
 [-0.02364636 -0.24432971 -0.0355177 ]
 [ 0.01372993 -0.01797578  0.00833985]
 [ 0.01568351  0.06921266 -0.07203457]
 [-0.08832397  0.01941863 -0.08670012]
 [ 0.06937144 -0.21037192  0.00277197]
 [-0.15536101 -0.09412256  0.06882738]
 [-0.06930019 -0.06716393  0.03574226]
 [ 0.1125401   0.06541362  0.17349062]
 [-0.0817717   0.02115303 -0.02008308]
 [-0.02294035 -0.12687331  0.0047435 ]
 [ 0.03979057 -0.17220323  0.01312203]
 [-0.03211477 -0.10454354 -0.01616936]
 [ 0.00773113 -0.12687228 -0.03173622]
 [ 0.0454241   0.2010085  -0.031517

In [9]:
import numpy as np

# Define the dimensions
N = 100  # Number of firms
L = 5    # Number of characteristics
T = 50   # Number of time periods
K = 3    # Number of factors (assumed based on the context)

# Generate random data for demonstration
np.random.seed(42)
Z = np.random.randn(T-1, N, L)  # Characteristics matrix (T-1 x N x L)
r = np.random.randn(T-1, N)     # Returns matrix (T-1 x N)

# Compute managed portfolio returns x_t+1 = Z_t' r_t+1 / N_t+1
X = np.zeros((T-1, L))  # T-1 x L
for t in range(T-1):
    X[t] = Z[t].T @ r[t] / N  # L x 1

# Compute the sample second moment matrix of managed portfolio returns
X_prime_X = X.T @ X  # L x L

# Initialize Gamma_beta as the first K eigenvectors of X'X
eigenvalues, eigenvectors = np.linalg.eig(X_prime_X)
Gamma_beta = eigenvectors[:, :K]  # L x K

# Ensure Gamma_beta is orthonormal (Gamma_beta' Gamma_beta = I_K)
Gamma_beta, _ = np.linalg.qr(Gamma_beta)

# Initialize f as random (T-1 x K)
f = np.random.randn(T-1, K)

def update_f(Gamma_beta, Z, r):
    f_new = np.zeros_like(f)  # T-1 x K
    for t in range(T-1):
        # Z[t] is N x L, Gamma_beta is L x K, r[t] is N x 1
        term1 = Gamma_beta.T @ Z[t].T @ Z[t] @ Gamma_beta  # K x K
        term2 = Gamma_beta.T @ Z[t].T @ r[t]  # K x 1
        f_new[t] = np.linalg.inv(term1) @ term2  # K x 1
    return f_new

def update_Gamma_beta(f, Z, r):
    term1 = np.zeros((L*K, L*K))  # L*K x L*K
    term2 = np.zeros(L*K)         # L*K x 1
    for t in range(T-1):
        # Z[t].T @ Z[t] is L x L, np.outer(f[t], f[t]) is K x K
        term1 += np.kron(Z[t].T @ Z[t], np.outer(f[t], f[t]))  # L*K x L*K
        # Z[t].T is L x N, f[t].T is 1 x K, r[t] is N x 1
        term2 += np.kron(Z[t], f[t].T).T @ r[t]  # L*K x 1
    Gamma_beta_new = np.linalg.inv(term1) @ term2  # L*K x 1
    Gamma_beta_new = Gamma_beta_new.reshape(L, K)  # Reshape to L x K

    # Ensure Gamma_beta_new is orthonormal (Gamma_beta' Gamma_beta = I_K)
    Gamma_beta_new, _ = np.linalg.qr(Gamma_beta_new)
    return Gamma_beta_new

# Alternating least squares
max_iter = 100
tolerance = 1e-6
for iteration in range(max_iter):
    f_new = update_f(Gamma_beta, Z, r)
    Gamma_beta_new = update_Gamma_beta(f_new, Z, r)

    # Check for convergence
    if np.linalg.norm(Gamma_beta_new - Gamma_beta) < tolerance:
        break
    Gamma_beta = Gamma_beta_new
    f = f_new

print(f"Converged after {iteration} iterations")
print("Estimated Gamma_beta:\n", Gamma_beta)
print("Estimated f:\n", f)

Converged after 73 iterations
Estimated Gamma_beta:
 [[-0.02742913 -0.08301632 -0.17273286]
 [-0.91003913 -0.23952883 -0.31052646]
 [ 0.09077497  0.08747398 -0.54295076]
 [ 0.2982179  -0.94334273 -0.11219341]
 [ 0.27184997  0.19541501 -0.75256481]]
Estimated f:
 [[ 0.09307144 -0.10053275  0.0378774 ]
 [-0.05713705  0.19834593  0.14539234]
 [-0.00041166 -0.08348312 -0.01204645]
 [ 0.12204416 -0.01798662 -0.00976616]
 [-0.00415178 -0.00166369  0.18541814]
 [-0.02520443  0.23835091 -0.03774222]
 [ 0.01671768  0.01863703  0.00886219]
 [ 0.01252518 -0.07577225 -0.07654618]
 [-0.10785679 -0.02794335 -0.09213025]
 [ 0.08418234  0.20857714  0.00294558]
 [-0.17392979  0.10011421  0.07313812]
 [-0.07682883  0.07009518  0.03798083]
 [ 0.13967859 -0.04732759  0.18435653]
 [-0.09623657 -0.02296353 -0.0213409 ]
 [-0.02401069  0.1260997   0.00504059]
 [ 0.04990358  0.17182516  0.01394388]
 [-0.03630116  0.10188761 -0.01718207]
 [ 0.00925812  0.12243132 -0.0337239 ]
 [ 0.04708243 -0.20219582 -0.033491